In [1]:
import csv 
import torch
import torch.utils.data
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import argparse
from torch.optim import Adam
from torch.autograd import Variable
from RecurrentNet import RecurrentNet




filepath2 = "D:\\MLdata\\tracks.csv"
data2 = pd.read_csv(filepath2, index_col=0, header=[0,1])
y_col = data2.loc[:,( 'track', 'genre_top')].replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
classes = set(y_col)
print(set(y_col))

{'Country', 'Instrumental', 'Old-Time / Historic', 'Electronic', 'International', 'Rock', 'Jazz', 'Spoken', 'Classical', 'Folk', 'Hip-Hop', 'Experimental', 'Soul-RnB', 'Easy Listening', 'Blues', 'Pop'}


In [2]:
import h5py
def read_hdf(file_path):
    return_dict = []
    with h5py.File(file_path, 'r') as hf:
        #print(list(hf.keys()))
        dataset = hf[file_path[9:-5]]
        #print(len(list(dataset.keys())))
        keys = list(dataset.keys())
        for i in keys:
            return_dict.append({
                "name":int(i.replace("-","/")[-10:-4]),
                "data":dataset[i][:]
            })
        return return_dict

In [3]:
from torch.optim import Adam
 
import argparse
import sys
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

class RecurrentNet(torch.nn.Module):
    def __init__(self, n_layers, seq_size, dropout, netType, n_inputs, n_outputs):
        super(RecurrentNet, self).__init__()
        self.n_layers = n_layers
        self.seq_size = seq_size
        if netType == 'LSTM':
            self.rec = nn.LSTM(n_inputs, seq_size, n_layers, dropout = dropout)
        if netType == 'GRU':
            self.rec = nn.GRU(n_inputs, seq_size, n_layers, dropout = dropout)
        self.relu = nn.ReLU()
        self.hidden1 = nn.Linear(seq_size, seq_size)
        self.hidden2 = nn.Linear(seq_size, int(seq_size/2))
        self.output = nn.Linear(int(seq_size/2), n_outputs)
       
        
    def forward(self, x):
        out, _ = self.rec(x) 
        out = self.relu(self.hidden1(out))
        out = self.relu(self.hidden2(out))
        out = self.output(out) 
        return out
        


In [4]:
import random

x_data = []


#print(type(data))
n_outputs = 0
for k in classes:
    try:
        x_data.extend(read_hdf("D:/MLdata/" + k + ".hdf5"))
        print(len(x_data))
        n_outputs += 1
    except:
        pass
random.shuffle(x_data)
print(x_data[0]['name'])
print(x_data[2]['name'])
print(x_data[65]['name'])
print(x_data[17]['name'])
print(n_outputs)
shape1 = len(x_data[0]['data'])
print((x_data[0]))
shape2 = len(x_data[0]['data'][0])
print(shape1)
print(shape2)

1000
1999
2999
3998
4998
5995
6994
7994
118891
853
73585
106567
8
{'name': 118891, 'data': array([[-38.719635, -58.235054, -53.233833, ..., -53.98723 , -53.17541 ,
        -51.428055],
       [-33.896454, -54.301975, -44.54223 , ..., -47.041042, -51.893463,
        -47.236473],
       [-28.294024, -33.407738, -33.531178, ..., -40.570015, -37.3398  ,
        -36.14995 ],
       ...,
       [-65.35934 , -56.610107, -44.424347, ..., -66.03724 , -61.50744 ,
        -64.33018 ],
       [-72.74361 , -63.917946, -47.802277, ..., -69.01964 , -64.23512 ,
        -69.64878 ],
       [-80.      , -80.      , -63.335968, ..., -79.2271  , -74.57732 ,
        -77.94803 ]], dtype=float32)}
128
646


In [5]:
y_data = []
x_values = []
for i in x_data:
    y_data.append(y_col[i['name']])
    #x_values.append(i['data'].flatten())
    x_values.append((i['data']).flatten())
print(y_data[300])
#print(x_values[300][-5:-1])
#print(x_values[350][-5:-1])
#print(x_values[10][-5:-1])
#print(x_values[1000][-5:-1])
#print(len(x_values))
#for k in range(20):
#    for j in range(20):
#        print(len(x_values[k][j]))
#print(len(x_values[0][0]))

Rock


In [6]:
print(len(x_values))
print(x_values[0])
print(len(x_values[0]))

7994
[-38.719635 -58.235054 -53.233833 ... -79.2271   -74.57732  -77.94803 ]
82688


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


label_encoder = LabelEncoder()
#x_col = x_data.to_numpy()
#print(len(y_col))
#print(len(x_col))
#print(len(x_col[0]))
#classes = set(y_col)
#print(len(classes))

#flatten x to normalize then unflatten
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_values)
print(x_scaled[1])
x_values = np.reshape(x_scaled, (len(x_values),shape1,shape2))

y_data = label_encoder.fit_transform(np.ravel(y_data))
y_data = y_data.reshape(-1, 1)
print(y_data)
encoder = OneHotEncoder(drop='first', sparse=False)
# transform data
onehot_y = encoder.fit_transform(y_data)
print(onehot_y)
#print(y_data)
X_train, X_test, y_train, y_test = train_test_split(x_values , onehot_y,  test_size=0.2, random_state=42)
print(X_train[3])
print(y_train[3])
X_train, X_test, y_train, y_test = tf.convert_to_tensor(X_train), tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_train), tf.convert_to_tensor(y_test)

[0.73451705 0.70670815 0.69438424 ... 0.         0.         0.08343067]
[[2]
 [4]
 [5]
 ...
 [3]
 [6]
 [7]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[0.59008293 0.62056904 0.63030612 ... 0.85129797 0.72675152 0.71318529]
 [0.60648689 0.70120623 0.70341125 ... 0.90419723 0.86546805 0.92969959]
 [0.65913885 0.66464627 0.65813429 ... 0.95820048 0.97592499 0.96154034]
 ...
 [0.28788214 0.2565033  0.         ... 0.45552707 0.35614147 0.30340624]
 [0.         0.         0.         ... 0.43655949 0.41584411 0.34932985]
 [0.         0.         0.         ... 0.28637819 0.25517778 0.17092018]]
[0. 0. 0. 0. 0. 0. 1.]


In [8]:
print(len(x_values))
print(x_values[0])
print(len(x_values[0]))
print(len(x_values[0][0]))

7994
[[0.51600456 0.27206182 0.33457708 ... 0.32515965 0.33530735 0.35714931]
 [0.57629433 0.32122531 0.44322214 ... 0.41198697 0.35133171 0.40954409]
 [0.64632471 0.58240328 0.58086028 ... 0.49287481 0.53325248 0.54812565]
 ...
 [0.18300829 0.29237366 0.44469566 ... 0.17453451 0.23115702 0.19587278]
 [0.09070492 0.20102568 0.40247154 ... 0.13725452 0.19706097 0.12939024]
 [0.         0.         0.2083004  ... 0.00966129 0.06778355 0.02564964]]
128
646


In [9]:
from torch.autograd import Variable
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()




def testAccuracy(model):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            audio, labels = data
            # run the model on the test set to predict labels
            outputs = model(audio)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    accuracy = (100 * accuracy / total)
    return(accuracy)



def train(model, num_epochs):
    
    best_accuracy = 0.0


    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (audio, labels) in enumerate(train_loader, 0):
            # zero the parameter gradients
            optimizer.zero_grad()
            
            outputs = model(audio)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()
            running_loss += loss.item()     # extract the loss value
            
        accuracy = testAccuracy()
        print('epoch:', epoch+1,' test accuracy: %d %%' % (accuracy))
        
net = RecurrentNet(n_layers = 3, seq_size = 5, dropout = 0.2, netType = 'LSTM', 
                   n_inputs = len(x_values[0]), n_outputs = n_outputs)


In [10]:
num_input = X_train.shape[1] - 1



train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader  = torch.utils.data.DataLoader(train_dataset,
                                 batch_size=train_dataset.__len__())



optimizer = torch.optim.Adam(net.parameters(),lr=args.lr,weight_decay=0.00001)

    # training loop
epoch = 0
count = 0
while epoch < args.epoch and count < 2000:
    epoch = epoch+1
    accuracy = train(net, train_loader, optimizer)
    if accuracy == 100:
        count = count+1
    else:
        count = 0

TypeError: 'numpy.int32' object is not callable

In [15]:
print(X_train[0][3])

tf.Tensor(
[0.45392098 0.53642281 0.87305622 0.919807   0.90854712 0.89576116
 0.80745203 0.59096985 0.399822   0.33548942 0.33107905 0.72995231
 0.6384782  0.64631622 0.55251147 0.53183699 0.43713722 0.7987641
 0.94692576 0.9002331  0.83657439 0.86084158 0.84373007 0.82863815
 0.80754285 0.81094761 0.80336809 0.77991283 0.7270967  0.70747182
 0.55035419 0.50194178 0.32349844 0.3278717  0.24007535 0.73611832
 0.9903044  0.88814662 0.81768794 0.84914468 0.86010885 0.85183873
 0.84212658 0.83039799 0.81253414 0.77431386 0.76249838 0.72717001
 0.62205486 0.57073793 0.43446999 0.4107261  0.47151752 0.64850912
 0.74312963 0.66387866 0.62960482 0.56069593 0.48941584 0.52906961
 0.64548564 0.52637811 0.48563375 0.40869226 0.47106519 0.58367348
 0.95003483 0.888026   0.86240106 0.88486583 0.87162278 0.85514894
 0.81588035 0.79895217 0.78648121 0.8547245  0.89500484 0.88597612
 0.74998658 0.62407045 0.50840063 0.46380663 0.5571198  0.63780463
 0.97781415 0.95768332 0.95126901 0.91500276 0.82159